In [2]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random
from google.colab import drive
from PIL import Image
from torch.nn import functional as F

def gen_split(root_dir, stackSize, train):
    DatasetRGB = []
    DatasetMMAP=[]
    Labels = []
    NumFrames = []
    if train==True:
      list_folder=['S1', 'S3', 'S4']
    elif train==False:
      list_folder=['S2']
    for dir_user in list_folder: #S1,S2,S3,S4
      class_id=0
      dir = os.path.join(root_dir, dir_user)
      for target in sorted(os.listdir(dir)): #action1, action2
        dir1 = os.path.join(dir, target)
        insts = sorted(os.listdir(dir1))
        if insts != []:
          i=0
          for inst in insts:
            inst_dir_rgb = os.path.join(dir1, inst, 'rgb')
            inst_dir_mmap = os.path.join(dir1, inst, 'mmaps')
            numFrames = len(glob.glob1(inst_dir_rgb, '*.png'))
            if numFrames >= stackSize: 
              DatasetRGB.append(inst_dir_rgb)
              DatasetMMAP.append(inst_dir_mmap)
              Labels.append(class_id)
              NumFrames.append(numFrames)
        class_id+=1
    return DatasetRGB, DatasetMMAP, Labels, NumFrames   


class makeDataset(Dataset):

    def __init__(self, root_dir, spatial_transform_rgb=None, spatial_transform_mmap=None, seqLen=20,
                 train=True, mulSeg=False, numSeg=1, fmt='.png', frameType='r', regression=False):

        self.frameType=frameType
        self.images, self.mmaps, self.labels, self.numFrames = gen_split(root_dir, 5, train)
        self.spatial_transform_rgb = spatial_transform_rgb
        self.spatial_transform_mmap = spatial_transform_mmap
        self.train = train
        self.mulSeg = mulSeg
        self.numSeg = numSeg
        self.seqLen = seqLen
        self.regression=regression
        self.fmt = fmt

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        vid_name_rgb = self.images[idx]
        vid_name_mmap = self.mmaps[idx]
        label = self.labels[idx]
        numFrame = self.numFrames[idx]
        regression=self.regression
        inpSeq = []
        inpSeq_mmap=[]
        self.spatial_transform_rgb.randomize_parameters()
        for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
            fl_name = vid_name_rgb + '/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
            img = Image.open(fl_name)
            img=self.spatial_transform_rgb(img.convert('RGB'))
            inpSeq.append(img)
            pos=0
            found=False
            while found==False:
              try:
                fl_name = vid_name_mmap + '/' + 'map' + str(int(np.floor(i+pos))).zfill(4) + self.fmt
                img = Image.open(fl_name)
                found=True
              except:
                try:
                  fl_name = vid_name_mmap + '/' + 'map' + str(int(np.floor(i-pos))).zfill(4) + self.fmt
                  img = Image.open(fl_name)
                  found=True
                except:
                  pos+=1
            if regression==False:
              img=self.spatial_transform_mmap(img.convert('1'))
            else:
              img=self.spatial_transform_mmap(img.convert('L'))
            inpSeq_mmap.append(img)
        inpSeq = torch.stack(inpSeq, 0)
        inpSeq_mmap=torch.stack(inpSeq_mmap,0)
        return inpSeq, inpSeq_mmap, label